# Практическое задание к уроку 3. Tensorflow

Постройте нейронную сеть (берем несложную полносвязную сеть, меняем число слоев, число нейронов, типы активации, тип оптимизатора) на датасете from sklearn.datasets import load_boston.
Постройте 10-15 вариантов разных нейронных сетей и сведите результаты их работы в таблицу. Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

In [1]:
from sklearn.datasets import load_boston

In [2]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
2.5.0-rc1


In [3]:
import pandas as pd

dataset = load_boston()
X = dataset["data"]
y = dataset["target"]
feature_names = np.append(dataset["feature_names"], ['target']) 

data = pd.DataFrame(data=np.c_[X, y], columns=feature_names)

In [4]:
data.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9


In [5]:
data.shape

(506, 14)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [7]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [8]:
inputs = Input(shape=(13))
x_1 = Dense(26, activation='relu')(inputs)
x_2 = Dense(26, activation='relu')(x_1)
x_2 = Dense(26, activation='relu')(x_1)
outputs = Dense(1)(x_2)

x_1, x_2, outputs

model = Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 26)                364       
_________________________________________________________________
dense_2 (Dense)              (None, 26)                702       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 27        
Total params: 1,093
Trainable params: 1,093
Non-trainable params: 0
_________________________________________________________________


In [9]:
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD

model.compile(optimizer=Adam(),
              loss="mse",
              metrics=[tf.keras.metrics.MeanSquaredError()])

In [10]:
def norm(x):
  mean = x.mean(axis=0)
  std = x.std(axis=0)
  return (x - mean) / std

In [11]:
X_train = norm(X_train)
X_test = norm(X_test)

In [12]:
history = model.fit(X_train, 
                    y_train, 
                    epochs=100,
                    validation_split=0.2, 
                    verbose=2)

Epoch 1/100
9/9 - 1s - loss: 634.0318 - mean_squared_error: 634.0318 - val_loss: 562.6766 - val_mean_squared_error: 562.6766
Epoch 2/100
9/9 - 0s - loss: 617.8674 - mean_squared_error: 617.8674 - val_loss: 547.4979 - val_mean_squared_error: 547.4979
Epoch 3/100
9/9 - 0s - loss: 602.2576 - mean_squared_error: 602.2576 - val_loss: 531.9523 - val_mean_squared_error: 531.9523
Epoch 4/100
9/9 - 0s - loss: 585.4257 - mean_squared_error: 585.4257 - val_loss: 515.3160 - val_mean_squared_error: 515.3160
Epoch 5/100
9/9 - 0s - loss: 567.4016 - mean_squared_error: 567.4016 - val_loss: 496.0649 - val_mean_squared_error: 496.0649
Epoch 6/100
9/9 - 0s - loss: 545.6177 - mean_squared_error: 545.6177 - val_loss: 473.8464 - val_mean_squared_error: 473.8464
Epoch 7/100
9/9 - 0s - loss: 520.2047 - mean_squared_error: 520.2047 - val_loss: 446.5796 - val_mean_squared_error: 446.5796
Epoch 8/100
9/9 - 0s - loss: 490.0525 - mean_squared_error: 490.0525 - val_loss: 413.4838 - val_mean_squared_error: 413.4838


In [14]:
# Делаем прогноз. Возвращается копия предсказания в виде одномерного массива
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest mse:', test_acc)

6/6 - 0s - loss: 25.0001 - mean_squared_error: 25.0001

Test mse: 25.00014877319336
